In [1]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install Pandas
import PyPDF2
import pandas as pd
import re

You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
data = {}
df = pd.DataFrame()

def extract_info(file):
    main_pdf = PyPDF2.PdfFileReader(file, 'rb')
    main_str = str()

    # decrypt pdf with "" as password
    if main_pdf.isEncrypted:
        main_pdf.decrypt("")

    for pages in range(main_pdf.getNumPages()):
        p = main_pdf.getPage(pages)
        p_text= p.extractText()
        p_lines=str(p_text.splitlines())
        if pages == 0:
            main_str += p_lines
        else:
            main_str += " \n "
            main_str += p_lines
            
    main_str.replace(" ", "")
    
    # Create dictionary to convert to dataframe later
    data = {
        # Change names if ASX changes their format
        'Entity Name' : (main_str.split("Entity name")[-1].split("Security on which the Distribution")[0]),
        'Ticker' : (main_str.split("Security on which the Distribution will be paid")[-1])[:3],
        'Record Date' : (main_str.split("2A.4 +Record Date")[-1].split("2A.5 Ex Date")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0]),
        'Payment Date' : (main_str.split("2A.6 Payment Date")[-1].split("2A.7 Are any of the")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0]),
        'Primary Currency' : (main_str.split("Currency in which the dividend/distribution is made")[-1][21:25]),
        'Dividend per Share' : (main_str.split("notified in this form")[-1][4:14]),
        "Dividend per Share Status" : "WILL UPDATE BELOWWWWW",
        "Franked Amount" : (main_str.split("3A.5")[0])[-10:],
        "Unfranked Amount" : (main_str.split("foreignincome amount")[-1])[4:14]
           }

    # update with whether dividends per share is actual or estimated
    if (main_str.split("estimated at this time?")[-1][:1]) == "Y":
        data.update({"Dividend per Share Status" : "Estimated"})
    else:
        data.update({"Dividend per Share Status" : "Actual"})

    # Sanity check on dividends
    if (data["Dividend per Share Status"]) == "Estimated":
        if (main_str.split("3A.1a(i)")[0])[-10:] == data["Dividend per Share"]:
            pass
        else:
            data.update({"Dividend per Share" : "ERROR GO FIND IT YOSELF"})
    else:
        if (main_str.split("3A.2")[0])[-10:] == data["Dividend per Share"]:
            pass
        else:
            data.update({"Dividend per Share" : "ERROR GO FIND IT YOSELF"})    
            
    df = pd.DataFrame.from_dict(data, orient='index', columns = [""])
    return df

extract_info('aad.pdf')


,
Entity Name,ARDENT LEISURE GROUP
Ticker,AAD
Record Date,"Tuesday June 26, 2018"
Ex Date,"Monday June 25, 2018"
Payment Date,"Friday August 31, 2018"
Primary Currency,AUD
Dividend per Share,0.06500000
Dividend per Share Status,Estimated
Franked Amount,0.00000000
Unfranked Amount,0.06500000
